In [1]:
from gen_ai_hub.proxy.langchain import init_llm
import pandas as pd
from hana_ml import dataframe
from hana_ai.tools.toolkit import HANAMLToolkit


connection_context = dataframe.ConnectionContext(userkey="RaysKey")

llm = init_llm('gpt-4-32k', temperature=0.0, max_tokens=1800)
toolkit = HANAMLToolkit(connection_context, used_tools='all', return_direct={"fetch_data": False})

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [mcp]4/5 [mcp]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.5 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [nltk]1/2 [nltk]


In [ ]:

toolkit.launch_mcp_server(server_name="test", sse_port=12345)

INFO:     Started server process [34700]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12345 (Press CTRL+C to quit)


INFO:     127.0.0.1:53211 - "GET /sse HTTP/1.1" 200 OK
INFO:     127.0.0.1:53213 - "POST /messages/?session_id=45b56cd21aa442d18d320925814c812f HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53213 - "POST /messages/?session_id=45b56cd21aa442d18d320925814c812f HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53213 - "POST /messages/?session_id=45b56cd21aa442d18d320925814c812f HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53213 - "POST /messages/?session_id=45b56cd21aa442d18d320925814c812f HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53376 - "GET /sse HTTP/1.1" 200 OK
INFO:     127.0.0.1:53378 - "POST /messages/?session_id=0ad5ed0738404898bda4a6a5793ec44a HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53378 - "POST /messages/?session_id=0ad5ed0738404898bda4a6a5793ec44a HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53378 - "POST /messages/?session_id=0ad5ed0738404898bda4a6a5793ec44a HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53378 - "POST /messages/?session_id=0ad5ed0738404898bda4a6a5793ec44a HTTP/1.1" 202 Acc

In [5]:
from mcp.client.sse import sse_client  # SSE协议客户端
from mcp import ClientSession  # MCP会话管理

async def get_mcp_tools():
    """Get MCP tools using SSE client."""
    try:
        # 1. start SSE client to connect to the MCP server
        async with sse_client("http://localhost:12345/sse") as streams:
            # 2. create a ClientSession with the streams
            async with ClientSession(*streams) as session:
                await session.initialize()  # 初始化会话
                
                # 3. get the list of tools
                tools = await session.list_tools()
                print(f"✅ successfully obtains {len(tools.tools)} tools:")
                
                # 4. print the tool names and descriptions
                for tool in tools.tools:
                    print(f"  🛠️ name: {tool.name}")
                    print(f"  📝 description: {tool.description}")

                print("✅ Successfully get tool list！")

                # 5. call the fetch_data tool
                try:
                    # call the fetch_data tool with correct arguments
                    print("🔍 calling fetch tool...")
                    result = await session.call_tool(name="fetch_data", arguments={
                        "kwargs": {
                            "table_name": "PAL_COVID_DATA_TBL",
                            "top_n": 5
                        }
                    })
                except Exception as e:
                    print(f"❌ Failed to load tool: {type(e).__name__}: {str(e)}")
                return result

                
    except Exception as e:
        print(f"❌ Failed to load tool: {str(e)}")
        return None

# Start the asynchronous function to get MCP tools
result = await get_mcp_tools()

✅ successfully obtains 17 tools:
  🛠️ name: accuracy_measure
  📝 description: To compute the accuracy measure using true and predict tables.
  🛠️ name: additive_model_forecast_fit_and_save
  📝 description: To fit the additive model forecast and save the model in model storage.
  🛠️ name: additive_model_forecast_load_model_and_predict
  📝 description: To load the additive model forecast from model storage and predict.
  🛠️ name: automatic_timeseries_fit_and_save
  📝 description: To fit an AutomaticTimeseries model and save it in the model storage.
  🛠️ name: automatic_timeseries_load_model_and_predict
  📝 description: To load a model and do the prediction using automatic timeseries model.
  🛠️ name: automatic_timeseries_load_model_and_score
  📝 description: To load a model and do the scoring for automatic timeseries.
  🛠️ name: cap_artifacts
  📝 description: To generate CAP artifacts for a given model from model storage. 
  🛠️ name: fetch_data
  📝 description: Fetch data from a given ta

In [6]:
print(result.content[0].text)

     Date  Confirmed  Recovered  Deaths  Increase rate
1/22/2020        555         28      17            NaN
1/23/2020        654         30      18      17.837838
1/24/2020        941         36      26      43.883792
1/25/2020       1434         39      42      52.391073
1/26/2020       2118         52      56      47.698745
